In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd

# Tải PhoBERT tokenizer và mô hình đã pre-trained
tokenizer = BertTokenizer.from_pretrained('vinai/phobert-base')
model = BertForSequenceClassification.from_pretrained('vinai/phobert-base', num_labels=2)  # 2 labels for binary classification

# Đọc dataset từ file CSV
data = pd.read_csv('/content/crawl_data.csv', encoding='utf-8') 

train_texts, test_texts, train_labels, test_labels = train_test_split(data['text'].astype(str).values, data['label'], test_size=0.2, random_state=42)

# Tokenize dữ liệu
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)

# Reset the index of train_labels and test_labels after the split
train_labels = train_labels.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

# Chuyển đổi thành torch Dataset
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Tạo dataset cho train và test
train_dataset = ReviewsDataset(train_encodings, train_labels)
test_dataset = ReviewsDataset(test_encodings, test_labels)

# Cài đặt các tham số huấn luyện
training_args = TrainingArguments(
    output_dir='./results',          # nơi lưu kết quả
    num_train_epochs=3,              # số lần huấn luyện qua dữ liệu
    per_device_train_batch_size=8,   # kích thước batch cho train
    per_device_eval_batch_size=16,   # kích thước batch cho test
    warmup_steps=500,                # bước warmup
    weight_decay=0.01,               # weight decay
    logging_dir='./logs',            # nơi lưu log
    logging_steps=10,
    evaluation_strategy="epoch",     # chiến lược đánh giá sau mỗi epoch
    save_strategy="epoch",           # lưu mô hình sau mỗi epoch
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,                         # mô hình PhoBERT
    args=training_args,                  # tham số huấn luyện
    train_dataset=train_dataset,         # dữ liệu huấn luyện
    eval_dataset=test_dataset,           # dữ liệu kiểm tra
)

# Fine-tune mô hình
trainer.train()

# Đánh giá mô hình
eval_results = trainer.evaluate()

# In kết quả đánh giá
print(eval_results)

e:\TOOLS\MyProjets\api_python\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
